In [ ]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine
import pandas as pd
import aux_functions as af

TypeError: 'DB_Connection' object is not callable

### RESUMEN DEL DÍA

In [23]:
# TREV

query = """ SELECT SUM(Actuals) AS Total_Actuals, SUM(OTB) AS Total_OTB, SUM(Pick_Up) AS Total_PickUp, SUM(Actuals + OTB + Forecast) AS TREV, (SUM(Actuals + OTB) * 1.0) / SUM(Actuals + OTB + Forecast) AS perc_expected_revenue FROM df_nh_demo;"""
pd.read_sql_query(query, con=engine)


,Total_Actuals,Total_OTB,Total_PickUp,TREV,perc_expected_revenue
0,1.344228e+08,32475008.0,8.819742e+08,1.722964e+08,0.968667


In [ ]:
# rrev
query = """SELECT 
    SUM(Actuals) AS Total_Actuals_RREV, 
    SUM(OTB) AS Total_OTB_RREV, 
    SUM(Pick_Up) AS Total_PickUp_RREV, 
    SUM(Actuals + OTB + Forecast) AS RREV, 
    (SUM(Actuals + OTB) * 1.0) / SUM(Actuals + OTB + Forecast) AS perc_expected_revenue_RREV 
FROM 
    df_nh_demo
WHERE 
    Metric = 'RP';
"""
pd.read_sql_query(query, con=engine)

,Total_Actuals_RREV,Total_OTB_RREV,Total_PickUp_RREV,RREV,perc_expected_revenue_RREV
0,96068823.89,25335987.0,3.561594e+08,1.230532e+08,0.986604


In [ ]:
# orev
query = """SELECT 
    SUM(Actuals) AS Total_Actuals_OREV, 
    SUM(OTB) AS Total_OTB_OREV, 
    SUM(Pick_Up) AS Total_PickUp_OREV, 
    SUM(Actuals + OTB + Forecast) AS OREV, 
    (SUM(Actuals + OTB) * 1.0) / SUM(Actuals + OTB + Forecast) AS perc_expected_revenue_OREV 
FROM 
    df_nh_demo
WHERE 
    Metric != 'RP';
"""
pd.read_sql_query(query, con=engine)

,Total_Actuals_RREV,Total_OTB_RREV,Total_PickUp_RREV,RREV,perc_expected_revenue_RREV
0,38354019.29,7139021.0,5.258148e+08,49243162.29,0.923845


In [ ]:
# porcentajes de rrev y orev

query = """SELECT 
    -- Total Revenue (TREV)
    SUM(Actuals + OTB + Forecast) AS TREV,
    
    -- Room Revenue (RREV)
    SUM(CASE WHEN Metric = 'RP' THEN Actuals + OTB + Forecast ELSE 0 END) AS RREV,
    
    -- Other Revenue (OREV)
    SUM(CASE WHEN Metric != 'RP' THEN Actuals + OTB + Forecast ELSE 0 END) AS OREV,
    
    -- Percentages
    (SUM(CASE WHEN Metric = 'RP' THEN Actuals + OTB + Forecast ELSE 0 END) * 1.0) / SUM(Actuals + OTB + Forecast) AS perc_RREV_of_TREV,
    (SUM(CASE WHEN Metric != 'RP' THEN Actuals + OTB + Forecast ELSE 0 END) * 1.0) / SUM(Actuals + OTB + Forecast) AS perc_OREV_of_TREV
FROM 
    df_nh_demo;"""
pd.read_sql_query(query, con=engine)

,TREV,RREV,OREV,perc_RREV_of_TREV,perc_OREV_of_TREV
0,1.722964e+08,1.230532e+08,49243162.29,0.714195,0.285805


In [ ]:
# porcentajes de actuals, otb y pick up 

query = """SELECT 
    -- TREV, RREV y OREV
    SUM(Actuals + OTB + Forecast) AS TREV,
    SUM(CASE WHEN Metric = 'RP' THEN Actuals + OTB + Forecast ELSE 0 END) AS RREV,
    SUM(CASE WHEN Metric != 'RP' THEN Actuals + OTB + Forecast ELSE 0 END) AS OREV,
    
    -- Porcentajes de Actuals
    (SUM(Actuals) * 1.0) / SUM(Actuals + OTB + Forecast) AS perc_Actuals_of_TREV,
    (SUM(CASE WHEN Metric = 'RP' THEN Actuals ELSE 0 END) * 1.0) / SUM(CASE WHEN Metric = 'RP' THEN Actuals + OTB + Forecast ELSE 0 END) AS perc_Actuals_of_RREV,
    (SUM(CASE WHEN Metric != 'RP' THEN Actuals ELSE 0 END) * 1.0) / SUM(CASE WHEN Metric != 'RP' THEN Actuals + OTB + Forecast ELSE 0 END) AS perc_Actuals_of_OREV,

    -- Porcentajes de OTB
    (SUM(OTB) * 1.0) / SUM(Actuals + OTB + Forecast) AS perc_OTB_of_TREV,
    (SUM(CASE WHEN Metric = 'RP' THEN OTB ELSE 0 END) * 1.0) / SUM(CASE WHEN Metric = 'RP' THEN Actuals + OTB + Forecast ELSE 0 END) AS perc_OTB_of_RREV,
    (SUM(CASE WHEN Metric != 'RP' THEN OTB ELSE 0 END) * 1.0) / SUM(CASE WHEN Metric != 'RP' THEN Actuals + OTB + Forecast ELSE 0 END) AS perc_OTB_of_OREV,

    -- Porcentajes de PickUp
    (SUM(Pick_Up) * 1.0) / SUM(Actuals + OTB + Forecast) AS perc_PickUp_of_TREV,
    (SUM(CASE WHEN Metric = 'RP' THEN Pick_Up ELSE 0 END) * 1.0) / SUM(CASE WHEN Metric = 'RP' THEN Actuals + OTB + Forecast ELSE 0 END) AS perc_PickUp_of_RREV,
    (SUM(CASE WHEN Metric != 'RP' THEN Pick_Up ELSE 0 END) * 1.0) / SUM(CASE WHEN Metric != 'RP' THEN Actuals + OTB + Forecast ELSE 0 END) AS perc_PickUp_of_OREV
FROM 
    df_nh_demo;
"""
pd.read_sql_query(query, con=engine)

,TREV,RREV,OREV,perc_Actuals_of_TREV,perc_Actuals_of_RREV,perc_Actuals_of_OREV,perc_OTB_of_TREV,perc_OTB_of_RREV,perc_OTB_of_OREV,perc_PickUp_of_TREV,perc_PickUp_of_RREV,perc_PickUp_of_OREV
0,1.722964e+08,1.230532e+08,49243162.29,0.780184,0.78071,0.77887,0.188483,0.205895,0.144975,5.118936,2.894352,10.677925


In [ ]:
# hotel BU y country
query1 = """SELECT 
    Hotel_BU, SUM(Actuals), SUM(OTB), SUM(Pick_Up)
FROM 
    df_nh_demo
GROUP BY Hotel_BU;
"""

query2 = """SELECT 
    Hotel_Country, SUM(Actuals), SUM(OTB), SUM(Pick_Up)
FROM 
    df_nh_demo
GROUP BY Hotel_Country;
"""

pd.read_sql_query(query2, con=engine)

,Hotel_Country,SUM(Actuals),SUM(OTB),SUM(Pick_Up)
0,Argentina,1877920.10,353394.0,1.797759e+08
1,Austria,3974529.80,834827.0,2.403335e+05
2,Belgium,4476469.69,1159382.0,1.825204e+05
3,Chile,572482.38,136311.0,3.604213e+07
4,Colombia,2104028.28,367831.0,5.299792e+08
5,Czech Republic,453232.74,171450.0,4.935598e+05
6,Denmark,1684484.79,309329.0,4.910860e+05
7,Ecuador,108357.85,11697.0,4.696950e+03
8,Finland,466744.22,53864.0,2.120126e+04
9,France,3439022.78,636191.0,1.913451e+05


In [ ]:
# total forecast por subbu
query = """
    SELECT 
        Hotel_SubBU AS Hotel_SubBU,  -- País o SubBU
        SUM(Forecast) AS Total_Forecast
    FROM 
        df_nh_demo
    GROUP BY 
        Hotel_SubBU
    ORDER BY
        Total_Forecast DESC;  """
    
pd.read_sql_query(query, con=engine)

,Hotel_SubBU,Total_Forecast
0,CE,1477115.0
1,BENELUX,1222747.0
2,SPAIN,787846.0
3,ITALY,768465.0
4,PORTUGAL,502535.0
5,SOUTHERN CONE,217132.0
6,FRANCE,191328.0
7,COLOMBIA,122960.0
8,MEXICO,108410.0


In [46]:
# métricas
query="""
-- Calcular el desempeño de todas las métricas disponibles
WITH Revenue_Metrics AS (
    SELECT 
        Metric,  -- Todas las métricas disponibles (ej. RP, BKF, FPB, etc.)
        
        -- Calcular los valores totales por cada métrica
        SUM(Actuals) AS Total_Actuals,
        SUM(OTB) AS Total_OTB,
        SUM(Pick_Up) AS Total_PickUp,
        SUM(Forecast) AS Total_Forecast,
        
        -- Cálculos de porcentajes para evaluar el desempeño
        (SUM(Pick_Up) * 1.0) / NULLIF(SUM(Forecast), 0) AS Perc_PickUp_to_Forecast,
        (SUM(OTB) * 1.0) / NULLIF(SUM(Forecast), 0) AS Perc_OTB_to_Forecast,
        (SUM(Actuals) * 1.0) / NULLIF(SUM(Forecast), 0) AS Perc_Actuals_to_Forecast,
        ((SUM(Actuals) + SUM(OTB)) * 1.0) / NULLIF(SUM(Forecast), 0) AS Perc_Actuals_OTB_to_Forecast
    FROM 
        df_nh_demo
    GROUP BY 
        Metric  -- Agrupamos solo por la métrica
)

-- Seleccionar y mostrar las métricas con sus respectivos cálculos
SELECT 
    Metric,
    Total_Actuals,
    Total_OTB,
    Total_PickUp,
    Total_Forecast,
    Perc_PickUp_to_Forecast,
    Perc_OTB_to_Forecast,
    Perc_Actuals_to_Forecast,
    Perc_Actuals_OTB_to_Forecast
FROM 
    Revenue_Metrics
ORDER BY 
    CASE 
        WHEN Metric = 'RP' THEN 1
        WHEN Metric = 'BKF' THEN 2
        WHEN Metric = 'FPB' THEN 3
        WHEN Metric = 'RREV' THEN 4
        ELSE 5  -- Ordenamos en el orden que prefieras o por defecto
    END;

 """

pd.read_sql_query(query, con=engine)

,Metric,Total_Actuals,Total_OTB,Total_PickUp,Total_Forecast,Perc_PickUp_to_Forecast,Perc_OTB_to_Forecast,Perc_Actuals_to_Forecast,Perc_Actuals_OTB_to_Forecast
0,RP,96068823.89,25335987.0,3.561594e+08,1648416.0,216.061584,15.369899,58.279478,73.649377
1,BKF,8802643.23,2104734.0,5.776501e+07,229787.0,251.385041,9.159500,38.307838,47.467338
2,EVENTS,3275378.70,967061.0,5.056256e+06,50122.0,100.878976,19.294142,65.348125,84.642267
3,F&B,21587365.96,3648648.0,3.678936e+08,2719176.0,135.295990,1.341821,7.938937,9.280758
4,Others,4688631.40,418578.0,9.509991e+07,751037.0,126.624795,0.557333,6.242877,6.800210
